# Práctica pySpark 

Datos abiertos Comunidad de Madrid - Censo de locales y actividades

In [2]:
import pandas as pd
import numpy as np


import findspark
findspark.init()

import pyspark

#Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ML_spark").getOrCreate()

In [3]:
import pyspark.sql.functions as pyf

In [4]:
spark

## EDA in Pyspark

In [5]:
#Read csv file without schema

df = df = spark.read.options(header=True, delimiter=';', quota='', inferSchema=True) \
    .csv('OPEN DATA locales-Epigrafes202010.csv')

df.printSchema()


root
 |-- id_local: string (nullable = true)
 |-- id_distrito_local: string (nullable = true)
 |-- desc_distrito_local: string (nullable = true)
 |-- id_barrio_local: integer (nullable = true)
 |-- desc_barrio_local: string (nullable = true)
 |-- cod_barrio_local: integer (nullable = true)
 |-- id_seccion_censal_local: string (nullable = true)
 |-- desc_seccion_censal_local: integer (nullable = true)
 |-- coordenada_x_local: string (nullable = true)
 |-- coordenada_y_local: string (nullable = true)
 |-- id_tipo_acceso_local: string (nullable = true)
 |-- desc_tipo_acceso_local: string (nullable = true)
 |-- id_situacion_local: string (nullable = true)
 |-- desc_situacion_local: string (nullable = true)
 |-- id_vial_edificio: integer (nullable = true)
 |-- clase_vial_edificio: string (nullable = true)
 |-- desc_vial_edificio: string (nullable = true)
 |-- id_ndp_edificio: integer (nullable = true)
 |-- id_clase_ndp_edificio: integer (nullable = true)
 |-- nom_edificio: string (nullable 

In [6]:
df.toPandas().head(3)

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,...,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe
0,285016136,8,FUENCARRAL-EL PARDO,803,PE�A GRANDE,NaN,None,157.0,"439038,63","4480515,56",...,None,PB,None,CENTRO DE YOGA IZEL,P,EDUCACI0N,85,EDUCACI0N,855002,"ENSE�ANZA NO REGLADA (DEPORTIVA Y RECREATIVA, ..."
1,285016137,8,FUENCARRAL-EL PARDO,806,VALVERDE,NaN,None,153.0,"441516,63","4483287,54",...,None,PB,None,SIN ACTIVIDAD,None,None,None,None,None,None
2,285016140,8,FUENCARRAL-EL PARDO,806,VALVERDE,NaN,None,153.0,"441515,63","4483304,54",...,None,PB,None,SIN ACTIVIDAD,None,None,None,None,None,None


In [7]:
df.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164445 entries, 0 to 164444
Data columns (total 46 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id_local                   164445 non-null  object 
 1   id_distrito_local          164445 non-null  object 
 2   desc_distrito_local        164445 non-null  object 
 3   id_barrio_local            164445 non-null  int32  
 4   desc_barrio_local          164445 non-null  object 
 5   cod_barrio_local           2 non-null       float64
 6   id_seccion_censal_local    2 non-null       object 
 7   desc_seccion_censal_local  164443 non-null  float64
 8   coordenada_x_local         164443 non-null  object 
 9   coordenada_y_local         164443 non-null  object 
 10  id_tipo_acceso_local       0 non-null       object 
 11  desc_tipo_acceso_local     164443 non-null  object 
 12  id_situacion_local         0 non-null       object 
 13  desc_situacion_local       16

El dataset contiene 46 columnas con 164.444 locales identificados. 

Vamos a hacer un pequeño análisis de los tipos de locales que hay en la comunidad de Madrid y su distribución por barrios


In [8]:
#Seleccionamos solo las columnas que vamos a necesitar para el análisis con select()

columns = ['id_local', 'desc_distrito_local', 'desc_barrio_local', 'desc_situacion_local', 'rotulo', 'desc_division', 'desc_epigrafe']

df = df.select(columns)
df.show(10)

+---------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
| id_local| desc_distrito_local|   desc_barrio_local|desc_situacion_local|             rotulo|       desc_division|       desc_epigrafe|
+---------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|285016136|FUENCARRAL-EL PARDO |PE�A GRANDE         |             Abierto|CENTRO DE YOGA IZEL|           EDUCACI0N|ENSE�ANZA NO REGL...|
|285016137|FUENCARRAL-EL PARDO |VALVERDE            |             Cerrado|      SIN ACTIVIDAD|                null|                null|
|285016140|FUENCARRAL-EL PARDO |VALVERDE            |             Cerrado|      SIN ACTIVIDAD|                null|                null|
|285016143|FUENCARRAL-EL PARDO |VALVERDE            |             Cerrado|      SIN ACTIVIDAD|                null|                null|
|285016146|FUENCARRAL-EL PARDO |VALVERDE 

In [9]:
#Limpiar los valores nulos que no aportan información con na.drop('any')

df.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164445 entries, 0 to 164444
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id_local              164445 non-null  object
 1   desc_distrito_local   164445 non-null  object
 2   desc_barrio_local     164445 non-null  object
 3   desc_situacion_local  164443 non-null  object
 4   rotulo                164443 non-null  object
 5   desc_division         123635 non-null  object
 6   desc_epigrafe         123635 non-null  object
dtypes: object(7)
memory usage: 8.8+ MB


In [10]:
df = df.na.drop('any')

In [11]:
#Comprobamos si hay Valores null

df.select([pyf.count(pyf.when(pyf.isnull(c), c)).alias(c) for c in df.columns]).show()

+--------+-------------------+-----------------+--------------------+------+-------------+-------------+
|id_local|desc_distrito_local|desc_barrio_local|desc_situacion_local|rotulo|desc_division|desc_epigrafe|
+--------+-------------------+-----------------+--------------------+------+-------------+-------------+
|       0|                  0|                0|                   0|     0|            0|            0|
+--------+-------------------+-----------------+--------------------+------+-------------+-------------+



In [12]:
#Distritos
df.select('desc_distrito_local').distinct().orderBy('desc_distrito_local').show()

+--------------------+
| desc_distrito_local|
+--------------------+
|ARGANZUELA          |
|BARAJAS             |
|CARABANCHEL         |
|CENTRO              |
|CHAMARTIN           |
|CHAMBERI            |
|CIUDAD LINEAL       |
|FUENCARRAL-EL PARDO |
|HORTALEZA           |
|LATINA              |
|MONCLOA-ARAVACA     |
|MORATALAZ           |
|PUENTE DE VALLECAS  |
|RETIRO              |
|SALAMANCA           |
|SAN BLAS-CANILLEJAS |
|TETUAN              |
|USERA               |
|VICALVARO           |
|VILLA DE VALLECAS   |
+--------------------+
only showing top 20 rows



In [13]:
#Numero de locales por distrito

df.groupBy('desc_distrito_local').count().orderBy('count', ascending=False).show()

+--------------------+-----+
| desc_distrito_local|count|
+--------------------+-----+
|CENTRO              |12443|
|CARABANCHEL         | 8881|
|SALAMANCA           | 8707|
|CIUDAD LINEAL       | 8004|
|CHAMBERI            | 7645|
|TETUAN              | 7497|
|PUENTE DE VALLECAS  | 7095|
|LATINA              | 6969|
|CHAMARTIN           | 6514|
|FUENCARRAL-EL PARDO | 6417|
|ARGANZUELA          | 5635|
|VILLAVERDE          | 5289|
|SAN BLAS-CANILLEJAS | 4939|
|HORTALEZA           | 4654|
|MONCLOA-ARAVACA     | 4529|
|USERA               | 4389|
|RETIRO              | 4146|
|VILLA DE VALLECAS   | 4070|
|MORATALAZ           | 2296|
|BARAJAS             | 1823|
+--------------------+-----+
only showing top 20 rows



In [14]:
#Situación y tipos de locales

df.groupBy('desc_situacion_local').count().show()

+--------------------+------+
|desc_situacion_local| count|
+--------------------+------+
|    Baja PC Asociado|    12|
|  Baja Reunificacion|  1809|
|             Cerrado|  4766|
|                Baja|   539|
|            En obras|    95|
|             Abierto|116342|
|        Uso vivienda|    72|
+--------------------+------+



In [21]:
df.groupBy('desc_division').count().orderBy(pyf.col('count').desc()).show()

+--------------------+-----+
|       desc_division|count|
+--------------------+-----+
|COMERCIO AL POR M...|43888|
|SERVICIOS DE COMI...|20945|
|OTROS SERVICIOS P...| 9786|
|           EDUCACI0N| 5108|
|ACTIVIDADES SANIT...| 3849|
|VENTA Y REPARACI0...| 3818|
|COMERCIO AL POR M...| 3033|
|ALMACENAMIENTO Y ...| 2405|
|ACTIVIDADES ADMIN...| 2370|
|ACTIVIDADES DE CO...| 2194|
|SERVICIOS FINANCI...| 2112|
|REPARACI0N DE ORD...| 1935|
|ACTIVIDADES DEPOR...| 1904|
|ACTIVIDADES ASOCI...| 1887|
|       SIN ACTIVIDAD| 1396|
|  TELECOMUNICACIONES| 1267|
|ACTIVIDADES INMOB...| 1066|
|ACTIVIDADES JURID...| 1009|
|ACTIVIDADES DE JU...|  888|
|PROMOCI0N Y CONST...|  749|
+--------------------+-----+
only showing top 20 rows



In [22]:
#Locales por distrito y situación

pivotdf = df.groupBy('desc_distrito_local').pivot('desc_situacion_local').count().orderBy('desc_distrito_local')
pivotdf.show()

+--------------------+-------+----+----------------+------------------+-------+--------+------------+
| desc_distrito_local|Abierto|Baja|Baja PC Asociado|Baja Reunificacion|Cerrado|En obras|Uso vivienda|
+--------------------+-------+----+----------------+------------------+-------+--------+------------+
|ARGANZUELA          |   5353|  23|            null|               114|    141|       1|           3|
|BARAJAS             |   1678|   1|            null|                31|    111|    null|           2|
|CARABANCHEL         |   8317|  77|               1|               165|    317|       2|           2|
|CENTRO              |  11719|  22|            null|               244|    445|      11|           2|
|CHAMARTIN           |   6167|  54|            null|                95|    165|      27|           6|
|CHAMBERI            |   7391|  38|               1|                61|    154|    null|        null|
|CIUDAD LINEAL       |   7594|  33|               1|                86|    281|   

In [68]:
#Cambiar el nombre de alguna columna

df = df.withColumnRenamed('desc_distrito_local', 'distrito') \
        .withColumnRenamed('desc_barrio_local', 'barrio') \
        .withColumnRenamed('rotulo', 'nombre_local')
df.printSchema()

root
 |-- id_local: string (nullable = true)
 |-- distrito: string (nullable = true)
 |-- barrio: string (nullable = true)
 |-- desc_situacion_local: string (nullable = true)
 |-- nombre_local: string (nullable = true)
 |-- desc_division: string (nullable = true)
 |-- desc_epigrafe: string (nullable = true)



In [76]:
#Nuevo dataframe solo con 1 distrito

df_retiro = df.filter(df.distrito == "RETIRO")
df_retiro.show()

+--------+--------+------+--------------------+------------+-------------+-------------+
|id_local|distrito|barrio|desc_situacion_local|nombre_local|desc_division|desc_epigrafe|
+--------+--------+------+--------------------+------------+-------------+-------------+
+--------+--------+------+--------------------+------------+-------------+-------------+



## Creamos una vista para hacer consultas sql

In [23]:
df.createOrReplaceTempView("df")

In [73]:
#Consulta
spark.sql('select * from df').show(3)

+---------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
| id_local| desc_distrito_local|   desc_barrio_local|desc_situacion_local|             rotulo|       desc_division|       desc_epigrafe|
+---------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|285016136|FUENCARRAL-EL PARDO |PE�A GRANDE         |             Abierto|CENTRO DE YOGA IZEL|           EDUCACI0N|ENSE�ANZA NO REGL...|
|285016158|FUENCARRAL-EL PARDO |PE�A GRANDE         |             Abierto|            MEZCLAX|OTROS SERVICIOS P...|SERVICIO DE PELUQ...|
|285016171|FUENCARRAL-EL PARDO |VALVERDE            |             Abierto|ROTULO NO INFORMADO|OTRAS INDUSTRIAS ...|FABRICACION DE AR...|
+---------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
only showing top 3 rows



In [74]:
#Consulta

spark.sql('select * from df where desc_distrito_local = "RETIRO"').show()

+--------+-------------------+-----------------+--------------------+------+-------------+-------------+
|id_local|desc_distrito_local|desc_barrio_local|desc_situacion_local|rotulo|desc_division|desc_epigrafe|
+--------+-------------------+-----------------+--------------------+------+-------------+-------------+
+--------+-------------------+-----------------+--------------------+------+-------------+-------------+



In [40]:
#Consulta usando group by

spark.sql('SELECT desc_distrito_local, count(*) \
            from df group by desc_distrito_local \
            order by desc_distrito_local').show()

+--------------------+--------+
| desc_distrito_local|count(1)|
+--------------------+--------+
|ARGANZUELA          |    5635|
|BARAJAS             |    1823|
|CARABANCHEL         |    8881|
|CENTRO              |   12443|
|CHAMARTIN           |    6514|
|CHAMBERI            |    7645|
|CIUDAD LINEAL       |    8004|
|FUENCARRAL-EL PARDO |    6417|
|HORTALEZA           |    4654|
|LATINA              |    6969|
|MONCLOA-ARAVACA     |    4529|
|MORATALAZ           |    2296|
|PUENTE DE VALLECAS  |    7095|
|RETIRO              |    4146|
|SALAMANCA           |    8707|
|SAN BLAS-CANILLEJAS |    4939|
|TETUAN              |    7497|
|USERA               |    4389|
|VICALVARO           |    1693|
|VILLA DE VALLECAS   |    4070|
+--------------------+--------+
only showing top 20 rows

